### Some things about bitwise operations.

x & y

Does a "bitwise and". Each bit of the output is 1 if the corresponding bit of x AND of y is 1, otherwise it's 0. 

example:

2&2 = 2
       
        2 ---> 0010
          and
        2 ---> 0010
              ------ 
               0010 = 2      
2&1 = 0
        
        2 ---> 0010
          and
        1 ---> 0001
              ------ 
               0000 = 1


x | y
    
Does a "bitwise or". Each bit of the output is 0 if the corresponding bit of x AND of y is 0, otherwise it's 1. 
    
2&1 = 3
        
        2 ---> 0010
           or
        1 ---> 0001
              ------ 
               0011 = 3


To have the octants (0-7) in binary numbers in mind: 

    0000 = 0        0001 = 1        0010 = 2         0011 = 3

    0100 = 4        0101 = 5        0110 = 6         0111 = 7


## Constructing the tree

Let's start to define the problem and our rules to build the tree. Consider there are $n=100$ particles randomly scatterd in the domain $x$, $y$, $z$ $\in$ $\left[ 0, 1 \right]$, each of them is a source and target. To contain all the particles, we define a **root** cubic cell centered at $(0.5,0.5, 0.5)$ with a side length of $1$. For a cubic cell, the radius is the half of side length, thus the root cell's radius $r_r$ is $0.5$. Then we choose $n_{crit}=10$ as the threshold to split a cell.

In [1]:
import numpy
from treecode_helper import Point, Particle

In [2]:
n = 100          # number of particles
particles = [ Particle(m=1.0/n) for i in range(n) ]

n_crit = 10      # max number of particles in a single cell

### Defining the class : Cell

Since each non-empty cell is an instance which has some same properties (eg. cell's center coordinates, its parent, its children, its multipole, particles inside if it's a leaf), we first need to define a class for a cell, and we call this class "Cell". Every cell is an instance of **Class** "**Cell**", and all the instances are stored in a **list** called "**cells**". Therefore, the root cell is "cells[0]". For those who have not been exposed to object-oriented programming in python, check [this](http://www.tutorialspoint.com/python/python_classes_objects.htm) out as a quick guide.

<img src="image/cell_class.png">

The figure above shows the "content" of a cell element:
* $x_c$, $y_c$, $z_c$, $r_c$: center coordinates and radius give the geometry of the cell
* a leaf of a cell is a particle stored in the cell, each leaf corresponds to a particle index from $0$ to $n-1$, and $n_{leaf}$ is number of leaves in the cell
* **parent** is the index (in the cells list) of the corresponding parent. 
* **child** is the array that contains the indices (in the cells list) of the corresponding children.
* **nchild** is an 8-bit binary number, each digit represents one of the eight child octants. $1$ denotes non-empty child cell, and $0$ denotes empty child cell in that octant. For example: nchild=00010100 means the current cell only has non-empty child in the fourth and sixth octant.
* **multipole**: array of $10$ multipole terms of the cell